In [1]:
# Import Necessary Libraries

import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.extractors import TitleExtractor
from llama_index.core import VectorStoreIndex
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings

from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper

import nest_asyncio
from diskcache import Cache

import getpass
import openai

In [2]:
# Apply nested asyncio to allow for nested event loops
nest_asyncio.apply()

# Ask user for OpenAI API key and set it
print("Enter your OpenAI API key: Please Note this is a demo version, only openai is currently supported. Input is hidden")
api_key = getpass.getpass()
openai.api_key = api_key

# Initialize a persistent Chroma database client with the specified path
chroma_client = chromadb.PersistentClient(path="./chroma.db")

# Define the path to the directory containing PDF documents to be processed
pdf_dir_path = "./dataset_folder"

# Set the language model and embedding model to be used for processing
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding()

# Initialize a cache to store results, with a specified cache directory
cache = Cache("./cache")
#Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

Enter your OpenAI API key: Please Note this is a demo version, only openai is currently supported. Input is hidden


In [3]:
def build_index(pdf_dir_path, storage_context):
    """Builds a vector store index from PDF documents.

    Args:
        pdf_dir_path (str): The path to the directory containing PDF documents to be indexed.
        storage_context (StorageContext): The context for storing the vector index.

    Returns:
        VectorStoreIndex: The created vector store index containing the processed documents.
    """  
    # Load documents from the specified directory
    docs = SimpleDirectoryReader(pdf_dir_path).load_data()
    
    # Create an ingestion pipeline with specified transformations
    pipeline = IngestionPipeline(
        transformations=[
            # Split documents into sentences with specified chunk size and overlap
            SentenceSplitter(chunk_size=2048, chunk_overlap=0),
            # Extract titles from the documents
            TitleExtractor(),
            # Use OpenAI's embedding model for text embedding
            OpenAIEmbedding(model_name="text-embedding-ada-002"),
            #HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2"),
        ]
    )
    
    # Run the pipeline on the loaded documents to create nodes
    nodes = pipeline.run(documents=docs)
    
    # Create a vector store index from the nodes and the provided storage context
    index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

    return index  # Return the created index

In [4]:

def data_retrieval(query, index):
    """Retrieves data based on the provided query from the specified index.

    Args:
        query (str): The query string used to search for relevant data.
        index (VectorStoreIndex): The index from which to retrieve data.

    Returns:
        list: A list of results matching the query.
    """
    # Convert the index into a retriever object for querying
    retriever = index.as_retriever()
    
    # Retrieve results based on the provided query
    results = retriever.retrieve(query)
    
    return results  # Return the retrieved results

In [5]:
def save_index():
    """Creates and saves a vector store index from PDF documents.

    This function attempts to create a new Chroma collection for storing the index.
    If the collection already exists, it retrieves the existing collection.
    It then builds the index from the documents in the specified directory and saves it.

    Returns:
        VectorStoreIndex or None: The created vector store index if successful, otherwise None.
    """
    print("Creating and saving the index")
    
    # Attempt to create a new Chroma collection for storing the index
    try:
        chroma_collection = chroma_client.create_collection(name="Insurance_Doc_RAG_LlamaIndex_LangChain")
    except Exception as e:
        # If the collection already exists, retrieve the existing collection
        print(f"Collection already exists: {e}")
        chroma_collection = chroma_client.get_collection(name="Insurance_Doc_RAG_LlamaIndex_LangChain")

    # Initialize a vector store with the Chroma collection
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create a storage context using the default settings and the vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Attempt to build the index from the documents and save it
    try:
        index = build_index(pdf_dir_path, storage_context)
        print("Index created and saved")
        return index  # Return the created index
    except Exception as e:
        # If an error occurs during index building, print the error and return None
        print(f"Error while building the index: {e}")
        return None

In [6]:
def load_index():
    """Loads the vector store index from a Chroma collection.

    This function attempts to load a vector store index from a specified Chroma collection.
    If the collection exists, it retrieves the collection and constructs a vector store index.
    If the collection does not exist or an error occurs, it returns None.

    Returns:
        VectorStoreIndex or None: The loaded vector store index if successful, otherwise None.
    """
    # Attempt to retrieve the specified collection from the Chroma database
    try:
        chroma_collection = chroma_client.get_collection(name="Insurance_Doc_RAG_LlamaIndex_LangChain")
    except Exception as e:
        # Print an error message if the collection cannot be loaded
        print(f"Error loading the collection: {e}")
        return None

    print("Loading the index")
    # Initialize a ChromaVectorStore with the retrieved collection
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    # Create a storage context using default settings and the vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    try:
        # Attempt to create a VectorStoreIndex from the vector store and storage context
        index = VectorStoreIndex.from_vector_store(
            vector_store=vector_store,
            storage_context=storage_context
        )
        return index  # Return the successfully loaded index
    except Exception as e:
        # Print an error message if the index cannot be loaded
        print(f"Error while loading the index: {e}")
        return None  # Return None if the index cannot be created

In [7]:
# Attempt to load the existing vector store index
index = load_index()

# If loading the index fails, attempt to create and save a new index
if index is None:
    index = save_index()

# Check if the index is successfully loaded or created
if index:
    print("Index is ready")  # Indicate that the index is ready for use
else:
    print("Failed to create or load the index")  # Indicate failure in loading or creating the index

Error loading the collection: Collection Insurance_Doc_RAG_LlamaIndex_LangChain does not exist.
Creating and saving the index


100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


Index created and saved
Index is ready


In [8]:
def retrieve_docs(query):
    """Retrieves documents based on the provided query, utilizing a cache for efficiency.

    This function first checks if the results for the given query are already cached. 
    If cached results are found, they are returned immediately. 
    If not, it retrieves the results from the index and caches them for future use.

    Args:
        query (str): The query string used to search for relevant documents.

    Returns:
        list: A list of results matching the query.
    """
    # Check if the results for the query are already in the cache
    if cache.get(query) is not None:
        extracted_texts = [result.node.text for result in cache.get(query)]
        return extracted_texts
        #return cache.get(query)  # Return cached results if available
    
    # Retrieve results from the index if not cached
    results = data_retrieval(query, index)
    
    # Store the retrieved results in the cache with a specified expiration time
    cache.set(query, results, expire=600)

    extracted_texts = [result.node.text for result in results]
    return extracted_texts  # Return the retrieved results

# Uncomment the following line to test the function with a sample query
# res = retrieve_docs("what is Waiting Period and Exclusions?")
# print(res)  # Uncomment to print the results

In [9]:
# Please Note - The following piece of code is not required to run the application and is used for internal understanding, alternate
# prototyping ways and testing/debugging purposes. This code is not required to run this application which the code used above/below has superseded. 


# from langchain.tools.retriever import create_retriever_tool

# doc_tool = create_retriever_tool(
#     retrive_docs,
#     "Search_docs",
#     "Search and return the relevant policy document data.",
# )

# from langchain_core.tools import tool
# from langchain.tools import Tool
# from langchain.prompts import PromptTemplate
# from langchain.memory import ConversationBufferMemory
# from langchain.chat_models import ChatOpenAI
# from langchain.agents import initialize_agent, AgentType
# from langchain.utilities import SerpAPIWrapper 

# from langchain.tools.retriever import create_retriever_tool




# # search_tool = Tool(
# #     name="Internet_Search",
# #     func=SerpAPIWrapper(serpapi_api_key="663d549846fac3c10e2b7d0dfeed509b0923c171084ecf2b8b4574bef5b3683a").run,
# #     description="Use this tool to search the internet for general information."
# # )

# # docs_tool = Tool(
# #     name="Cache and Document Search",
# #     func=retrive_docs,
# #     description="Use this tool to answer questions about policy documents."
# # )



# prompt_template = PromptTemplate.from_template("""
# You are a question answering expert. The user will ask you a question/query. Answer the question to the best of your ability.
# You have access to the following tools:

# {tools}

# Use the following format to interact with tools:
# Thought: What you are thinking.
# Action: tool_name("input")
# Observation: The result from the tool.

# Always use the Search_docs tool first to find answers in insurance documents. If you can't find a satisfactory answer, use the Internet_Search tool.

# Available tools: {tool_names}

# Current chat history:
# {chat_history}

# Agent Scratchpad:
# {agent_scratchpad}

# Intermediate Steps:
# {intermediate_steps}

# Input: {input}
# """)


# #prompt = PromptTemplate.from_template(prompt_template, template_format="jinja2")
# # memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# # tools = [doc_tool, search_tool]
# # llm = ChatOpenAI(model="gpt-4o-mini")
# # agent = initialize_agent(tools = tools, 
# #     llm = llm, 
# #     prompt=custom_prompt, 
# #     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
# #     verbose=True, 
# #     memory=memory)


# from langgraph.prebuilt import create_react_agent

# langgraph_agent_executor = create_react_agent(model, tools)
# message_history = messages["messages"]

# new_query = "What is dental work?"

# messages = langgraph_agent_executor.invoke(
#     {"messages": message_history + [("human", new_query)]}
# )
# {
#     "input": new_query,
#     "output": messages["messages"][-1].content,
# }

# #%pip install langchain-openai

# from langchain_openai import OpenAI
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain_core.messages import AIMessage, HumanMessage

# memory = ConversationBufferMemory(memory_key="chat_history")
# model = OpenAI(name='gpt-4o-mini',temperature=0)
# tools = [doc_tool, search_tool]

# agent = create_react_agent(model, tools, prompt_template)


# agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, handle_parsing_errors=True)

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Initialize the language model with the specified model and API key
model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

@tool
def doc_search_tool(query: str): 
    """Searches the documents for relevant information.

    Args:
        query (str): The query string to search within the documents.

    Returns:
        list: A list of relevant document excerpts matching the query.
    """
    return retrieve_docs(query)

@tool
def internet_search_tool(query: str):
    """Searches the internet for general information.

    Args:
        query (str): The query string to search on the internet.

    Returns:
        str: The result of the internet search.
    """
    return SerpAPIWrapper(serpapi_api_key="663d549846fac3c10e2b7d0dfeed509b0923c171084ecf2b8b4574bef5b3683a").run(query)

# List of tools available for the agent to use
tools = [doc_search_tool, internet_search_tool]

# Example query to demonstrate the agent's functionality
query = "What is the procedure to claim the insurance?"

# Define the prompt template for the chat agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a question answering expert. The user will ask you a question/query. Answer the question to the best of your ability
         Only use the Search_docs tool first to find answers in insurance documents. If you can't find a satisfactory answer, use the Internet_Search tool.
         """),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Create the agent using the model, tools, and prompt
agent = create_tool_calling_agent(model, tools, prompt)

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)

# Uncomment the following line to invoke the agent with the example query
# agent_executor.invoke({"input": query})

In [11]:
# Add a thread ID to the configuration for saving the conversation.
# This ensures that each conversation is saved and context is retained.

config = {"configurable" : {"thread_id" : "2"}}

In [13]:
# Welcome message for the user and instructions on how to exit the chatbot
print("Welcome to Insurance Documentation Chatbot. Please enter your query. Type 'exit' to quit.")

# Initialize an empty list to store the conversation history
message_history = []

# Continuously prompt the user for input until they type 'exit'
query = input("User: ")
while query.lower() != "exit":
    # Display the user's query
    print("User: ", query)
    print("Searching... Please wait!")
    print("-" * 100)
    
    # Combine the message history with the current query for context
    combined_messages = message_history + [("human", query)]
    
    # Invoke the agent executor with the combined messages to get a response
    response = agent_executor.invoke({"input": combined_messages})
    
    # Create a dictionary to store the input and output of the current interaction
    messages = {
        "input": query,
        "output": response['output'],
    }
    
    # Display the chatbot's response
    print("Chatbot: ", response['output'])
    print("-" * 100)
    
    # Append the current interaction to the message history
    message_history.append(messages)
    
    # Prompt the user for the next query
    query = input("User: ")

# Thank the user for using the chatbot when they exit
if 'exit' in query.lower():
    print("Thanks for using Insurance Documentation Chatbot. Have a great day!")
    print("-" * 100)

Welcome to Insurance Documentation Chatbot. Please enter your query. Type 'exit' to quit.
User:  what is Waiting Period and Exclusions?
Searching... Please wait!
----------------------------------------------------------------------------------------------------
Chatbot:  ### Waiting Period
The **Waiting Period** refers to a specified duration during which no benefits will be paid if a covered event occurs. For example, in the context of the Accelerated Critical Illness Benefit, there is a **90 Days Waiting Period**. This means that if a Scheme Member is diagnosed with any applicable listed Critical Illness or undergoes surgery within 90 days from the commencement of the coverage term, no benefits will be payable. However, if the Critical Illness is a result of an accident (such as Major Head Trauma), this waiting period may not apply.

### Exclusions
**Exclusions** are specific conditions or circumstances under which the insurance policy will not pay benefits. Key exclusions include:
